In [1794]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
import pickle

In [1795]:
mobile_dataset=pd.read_csv('train.csv')
testing_dataset=pd.read_csv("test.csv")


In [1796]:
mobile_dataset.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [1797]:
mobile_dataset.price_range.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [1798]:
testing_dataset.isna().sum()

id               0
battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
dtype: int64

In [1799]:
mobile_dataset.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [1800]:
mobile_dataset.duplicated().sum()

0

In [1801]:
mobile_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [1802]:

import numpy as np


mobile_dataset['sc_size'] = np.sqrt((mobile_dataset['sc_h']**2) + (mobile_dataset['sc_w']**2))
mobile_dataset['sc_size'] = round(mobile_dataset['sc_size']/2.54, 2)


In [1803]:
testing_dataset['sc_size'] = np.sqrt((testing_dataset['sc_h']**2) + (testing_dataset['sc_w']**2))
testing_dataset['sc_size'] = round(testing_dataset['sc_size']/2.54, 2)

In [1804]:
mobile_dataset.drop(['sc_h', 'sc_w'], axis = 1, inplace = True)

In [1805]:
mobile_dataset['pixels'] = mobile_dataset['px_height']*mobile_dataset['px_width']

In [1806]:
testing_dataset['pixels'] =testing_dataset['px_height']*testing_dataset['px_width']

In [1807]:
mobile_dataset.drop(['px_height', 'px_width'], axis = 1, inplace = True)

In [1808]:
mobile_dataset.price_range.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [1809]:
X = mobile_dataset.drop(columns=['price_range'])  
y = mobile_dataset['price_range'] 



In [1810]:
print(X.head())

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  pc   ram  talk_time  three_g  touch_screen  wifi  \
0        188        2   2  2549         19        0             0     1   
1        136        3   6  2631          7        1             1     0   
2        145        5   6  2603          9        1             1     0   
3        131        6   9  2769         11        1             0     0   
4        141        2  14  1411         15        1             1     0   

   sc_size   pixels  
0     4.49    15120  
1     6.80  1799140  
2 

In [2129]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=3,stratify=y)
                                                                          



In [2130]:
X_test

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,ram,talk_time,three_g,touch_screen,wifi,sc_size,pixels
939,1589,0,2.6,0,9,0,46,0.4,84,3,16,2373,20,1,1,0,5.60,580920
1026,606,0,2.5,0,1,1,53,0.2,146,4,2,1223,7,1,0,0,8.85,535680
1007,1963,1,1.0,0,16,1,34,0.1,157,2,17,714,2,1,0,1,7.13,2059568
213,1827,0,1.7,0,4,1,39,0.5,128,5,16,2844,10,1,1,0,6.55,56100
154,1547,0,2.9,1,6,0,45,0.7,166,2,8,505,3,0,1,1,4.79,343970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,581,0,2.2,0,0,0,59,0.7,185,5,6,2819,9,0,0,1,10.04,1295239
1286,584,1,1.8,0,0,1,19,0.3,168,3,0,465,4,1,1,0,5.68,1821506
132,645,0,0.5,0,1,0,41,1.0,197,7,10,2962,4,1,1,1,5.91,189660
1016,551,1,2.8,0,0,1,54,0.1,172,7,15,1414,19,1,0,1,2.39,323804


In [2131]:
selector = SelectKBest(score_func=chi2, k=10)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)


In [2132]:
selected_indices = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_indices]

In [2133]:
selected_feature_names

Index(['battery_power', 'fc', 'int_memory', 'mobile_wt', 'n_cores', 'pc',
       'ram', 'talk_time', 'sc_size', 'pixels'],
      dtype='object')

In [2134]:
testing_dataset1 = testing_dataset[['battery_power', 'fc', 'int_memory', 'mobile_wt','n_cores', 'pixels',
                                    'sc_size', 'ram', 'pc', 'talk_time']]

In [2135]:
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

In [2136]:
scaler = MinMaxScaler()


In [2137]:
X_train_scaled = scaler.fit_transform(X_train_selected)


In [2138]:
X_test_scaled = scaler.transform(X_test_selected)

In [2139]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


In [2140]:
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [2141]:
y_pred_rf = model_rf.predict(X_test_scaled)


In [2142]:
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf}")

Random Forest Accuracy: 0.8875


In [2143]:
from sklearn.metrics import classification_report


print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       100
           1       0.83      0.79      0.81       100
           2       0.87      0.85      0.86       100
           3       0.95      0.97      0.96       100

    accuracy                           0.89       400
   macro avg       0.89      0.89      0.89       400
weighted avg       0.89      0.89      0.89       400



In [2144]:
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest MSE: {mse_rf}")



Random Forest MSE: 0.1125


In [2145]:
with open('model_rf.pkl', 'wb') as file:
    pickle.dump(model_rf, file)


In [2146]:
model_lr = LogisticRegression()
model_lr.fit(X_train_scaled, y_train)


LogisticRegression()

In [2147]:
y_pred_lr = model_lr.predict(X_test_scaled)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {accuracy_lr}")


Logistic Regression Accuracy: 0.885


In [2148]:
from sklearn.metrics import classification_report


print(classification_report(y_test,y_pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94       100
           1       0.88      0.83      0.86       100
           2       0.85      0.80      0.82       100
           3       0.89      0.94      0.91       100

    accuracy                           0.89       400
   macro avg       0.88      0.88      0.88       400
weighted avg       0.88      0.89      0.88       400



In [2149]:

mse_lr = mean_squared_error(y_test, y_pred_lr)
print(f"Logistic Regression MSE: {mse_lr}")

Logistic Regression MSE: 0.115


In [2150]:

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100], 
    'penalty': ['l1', 'l2'],  
    'solver': ['liblinear', 'saga']  
}

grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(X_train_scaled, y_train)


print("Best parameters found: ", grid_search.best_params_)


lr_gridsearch = grid_search.best_estimator_


/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

Best parameters found:  {'C': 100, 'penalty': 'l1', 'solver': 'saga'}


/Users/sahilsingh/.pyenv/versions/3.11.0/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [2151]:
from sklearn.metrics import accuracy_score, classification_report

y_pred_lr_gridsearch = lr_gridsearch.predict(X_test_scaled)


accuracy_lr_gridsearch = accuracy_score(y_test, y_pred_lr_gridsearch)
print("Accuracy:", accuracy_lr_gridsearch)


print("Classification Report:")
print(classification_report(y_test, y_pred_lr_gridsearch))


Accuracy: 0.925
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       100
           1       0.91      0.89      0.90       100
           2       0.93      0.87      0.90       100
           3       0.92      0.99      0.95       100

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.92       400
weighted avg       0.93      0.93      0.92       400



In [2152]:
lr_gridsearch_mse=mean_squared_error(y_pred_lr_gridsearch,y_test)

In [2153]:
with open('model_lr_gridsearch.pkl', 'wb') as f:
    pickle.dump(lr_gridsearch, f)

In [2154]:
with open('model_lr.pkl', 'wb') as file:
    pickle.dump(model_lr, file)

In [2155]:
model_svc = SVC()
model_svc.fit(X_train_scaled, y_train)


SVC()

In [2156]:
y_pred_svc = model_svc.predict(X_test_scaled)
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"Support Vector Machine Accuracy: {accuracy_svc}")

Support Vector Machine Accuracy: 0.8725


In [2157]:
mse_svc = mean_squared_error(y_test, y_pred_svc)
print(f"Support Vector Machine MSE: {mse_svc}")

Support Vector Machine MSE: 0.1275


In [2158]:
param_grid = {
    'C': [0.1, 1, 10, 100],  
    'gamma': [1, 0.1, 0.01, 0.001], 
    'kernel': ['linear', 'rbf', 'poly'] 
}

svc_grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)

svc_grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'poly']})

In [2159]:
best_model_svc = svc_grid_search.best_estimator_

y_pred_svc_gridsearch = best_model_svc.predict(X_test_scaled)

print( classification_report(y_test, y_pred_svc_gridsearch))


              precision    recall  f1-score   support

           0       0.95      0.94      0.94       100
           1       0.91      0.89      0.90       100
           2       0.93      0.90      0.91       100
           3       0.93      0.99      0.96       100

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400



In [2160]:
accuracy_gridsearch_svc=accuracy_score(y_pred_svc_gridsearch,y_test)

In [2161]:
mse_gridsearch_svc=mean_squared_error(y_pred_svc_gridsearch,y_test)

In [2162]:
with open('modelgridsearch_svc.pkl', 'wb') as file:
    pickle.dump(best_model_svc, file)

In [2163]:
with open('model_svc.pkl', 'wb') as file:
    pickle.dump(model_svc, file)

In [2164]:
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train_scaled, y_train)



DecisionTreeClassifier()

In [2165]:
y_pred_dt = model_dt.predict(X_test_scaled)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Decision Tree Accuracy: {accuracy_dt}")

Decision Tree Accuracy: 0.84


In [2166]:
mse_dt = mean_squared_error(y_test, y_pred_dt)

print(f"Decision Tree MSE: {mse_dt}")

Decision Tree MSE: 0.1675


In [2167]:
with open('model_dt.pkl', 'wb') as file:
    pickle.dump(model_dt, file)

In [2168]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train_scaled, y_train)


GradientBoostingClassifier()

In [2169]:
y_pred_gb = model_gb.predict(X_test_scaled)
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb}")

Gradient Boosting Accuracy: 0.8775


In [2170]:
mse_gb = mean_squared_error(y_test, y_pred_gb)

print(f"Gradient Boosting MSE: {mse_gb}")

Gradient Boosting MSE: 0.1225


In [2171]:


with open('model_gb.pkl', 'wb') as file:
    pickle.dump(model_gb, file)

In [2172]:
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)


GaussianNB()

In [2173]:
y_pred_nb = model_nb.predict(X_test_scaled)
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Gaussian Naive Bayes Accuracy: {accuracy_nb}")

Gaussian Naive Bayes Accuracy: 0.8


In [2174]:
mse_nb = mean_squared_error(y_test, y_pred_nb)
print(f"Gaussian Naive Bayes MSE: {mse_nb}")

Gaussian Naive Bayes MSE: 0.2


In [2175]:
with open('model_nb.pkl', 'wb') as file:
    pickle.dump(model_nb, file)

In [2176]:
mse_data = {
    'Model': ['Random Forest', 'Logistic Regression', 'Support Vector Machine', 
              'Decision Tree', 'Gradient Boosting', 'Gaussian Naive Bayes',"lr_gridsearch","svc_grid_search"],
    'MSE': [mse_rf, mse_lr, mse_svc, mse_dt, mse_gb, mse_nb,lr_gridsearch_mse,mse_gridsearch_svc]
}

mse_df = pd.DataFrame(mse_data)
print(mse_df)

                    Model     MSE
0           Random Forest  0.1125
1     Logistic Regression  0.1150
2  Support Vector Machine  0.1275
3           Decision Tree  0.1675
4       Gradient Boosting  0.1225
5    Gaussian Naive Bayes  0.2000
6           lr_gridsearch  0.0750
7         svc_grid_search  0.0700


In [2177]:

accuracy_data = {
    'Model': ['Random Forest', 'Logistic Regression', 'Support Vector Machine', 
              'Decision Tree', 'Gradient Boosting', 'Gaussian Naive Bayes','lr_gridsearch',svc_grid_search],
    'Accuracy': [accuracy_rf, accuracy_lr, accuracy_svc, accuracy_dt, accuracy_gb, accuracy_nb,accuracy_lr_gridsearch,accuracy_gridsearch_svc]
}

accuracy_df = pd.DataFrame(accuracy_data)


print(accuracy_df)


                                               Model  Accuracy
0                                      Random Forest    0.8875
1                                Logistic Regression    0.8850
2                             Support Vector Machine    0.8725
3                                      Decision Tree    0.8400
4                                  Gradient Boosting    0.8775
5                               Gaussian Naive Bayes    0.8000
6                                      lr_gridsearch    0.9250
7  GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,...    0.9300


In [2178]:

mse_df_sorted = mse_df.sort_values(by='MSE')
best_model = mse_df_sorted.iloc[0]['Model']

print(f"The best model based on MSE is: {best_model}")


The best model based on MSE is: svc_grid_search
